In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import re
import pandas as pd

### Obteniedo metadatos cursos

In [2]:
columns = ['title', 'description', 'difficulty_level', 'duration', 'keywords', 'url']
df = pd.DataFrame(columns = columns)

In [3]:
def get_metadata_page(url): 
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    h1_tags = soup('h1')
    title = h1_tags[0].get_text()

    description = soup.find('div', {'class': 'content-inner'}).get_text()

    h4_tags = soup('h4')
    difficulty_level = ''
    duration = ''
    language = h4_tags[-1].get_text()
    for h4 in h4_tags:    
        if (re.search("Level", str(h4))):
            difficulty_level = h4.get_text()
        if (re.search("Approx", str(h4))):
            duration = h4.get_text()

    entradas = soup.find('div', {'class': 'Box_120drhm-o_O-displayflex_poyjc-o_O-wrap_rmgg7w'})
    keywords = ''
    for span in entradas:
        keywords += span.get_text() + ', '

    diccionario = {'title': title, 'description': description, 'difficulty_level': difficulty_level,
                   'duration': duration, 'keywords': keywords, 'url': url}
    return diccionario

### Iterar sobre todos los cursos

In [7]:
# Solo cursos en ingles
url = 'https://www.coursera.org/learn/neural-networks-deep-learning'
#url = 'https://www.coursera.org/learn/machine-learning'

diccionario = get_metadata_page(url)
df = df.append(diccionario, ignore_index=True)

In [8]:
df

,title,description,difficulty_level,duration,keywords,url
0,Machine Learning,Machine learning is the science of getting com...,,Approx. 56 hours to complete,"Logistic Regression, Artificial Neural Network...",https://www.coursera.org/learn/machine-learning
1,Neural Networks and Deep Learning,"If you want to break into cutting-edge AI, thi...",Intermediate Level,Approx. 18 hours to complete,"Artificial Neural Network, Backpropagation, Py...",https://www.coursera.org/learn/neural-networks...


In [9]:
df.to_json("metadataCoursera.json", orient='records')